In [1]:
import sys
import os
sys.path.append(os.path.abspath(os.path.join(os.getcwd(), '../..')))

# Path Configuration
from tools.preprocess import *

# Processing context
trait = "Endometrioid_Cancer"
cohort = "GSE73637"

# Input paths
in_trait_dir = "../../input/GEO/Endometrioid_Cancer"
in_cohort_dir = "../../input/GEO/Endometrioid_Cancer/GSE73637"

# Output paths
out_data_file = "../../output/preprocess/Endometrioid_Cancer/GSE73637.csv"
out_gene_data_file = "../../output/preprocess/Endometrioid_Cancer/gene_data/GSE73637.csv"
out_clinical_data_file = "../../output/preprocess/Endometrioid_Cancer/clinical_data/GSE73637.csv"
json_path = "../../output/preprocess/Endometrioid_Cancer/cohort_info.json"


### Step 1: Initial Data Loading

In [2]:
from tools.preprocess import *
# 1. Identify the paths to the SOFT file and the matrix file
soft_file, matrix_file = geo_get_relevant_filepaths(in_cohort_dir)

# 2. Read the matrix file to obtain background information and sample characteristics data
background_prefixes = ['!Series_title', '!Series_summary', '!Series_overall_design']
clinical_prefixes = ['!Sample_geo_accession', '!Sample_characteristics_ch1']
background_info, clinical_data = get_background_and_clinical_data(matrix_file, background_prefixes, clinical_prefixes)

# 3. Obtain the sample characteristics dictionary from the clinical dataframe
sample_characteristics_dict = get_unique_values_by_row(clinical_data)

# 4. Explicitly print out all the background information and the sample characteristics dictionary
print("Background Information:")
print(background_info)
print("Sample Characteristics Dictionary:")
print(sample_characteristics_dict)


Background Information:
!Series_title	"Prior Knowledge Transfer Across Transcriptional Datasets Using Compositional Statistics [Cell lines]"
!Series_summary	"Compositional statistics and random gene-sets were used to assign the tumor site of origin and histopathology of 18 epithelial ovarian cancer cell lines"
!Series_overall_design	"In the study presented here, we applied Gene Expression Compositional Assignment (GECA) to epithelial ovarian cell lines, using first a reference library of solid tumors (expO [http://www.intgen.org/expo/]) and then a second library of expert pathologically-reviewed epithelial ovarian cancer samples (GSE73551)"
Sample Characteristics Dictionary:
{0: ['cell type: ovarian cells'], 1: ['cell line: COV504', 'cell line: COV362', 'cell line: UWB1.289+BRCA1', 'cell line: OV56', 'cell line: UWB1.289', 'cell line: COV318', 'cell line: NCI/ADR-RES', 'cell line: OVCAR3', 'cell line: OVCAR4', 'cell line: OVCAR8', 'cell line: IGR-OV1', 'cell line: SK-OV-3', 'cell line:

### Step 2: Dataset Analysis and Clinical Feature Extraction

In [3]:
# 1. Evaluate gene expression data availability
# This dataset appears to be gene expression data from cell lines, not miRNA or methylation data
is_gene_available = True

# 2. Variable Availability and Data Type Conversion
# 2.1 Find keys for trait, age, and gender
# For trait (Endometrioid Cancer), we can use histopathology (key 3)
trait_row = 3

# No age information is available in the sample characteristics
age_row = None

# No gender information is available (these are cell lines)
gender_row = None

# 2.2 Define conversion functions for each variable
def convert_trait(value):
    """Convert histopathology information to binary trait values for Endometrioid Cancer"""
    if value is None:
        return None
    
    # Extract value after the colon if present
    if ':' in value:
        value = value.split(':', 1)[1].strip()
    
    # Convert to binary: 1 for Endometrioid, 0 for others
    if 'Endometrioid' in value or 'Endometroid' in value:  # Account for possible spelling variation
        return 1
    else:
        return 0

def convert_age(value):
    """Convert age information - not available for this dataset"""
    return None

def convert_gender(value):
    """Convert gender information - not available for this dataset"""
    return None

# 3. Save metadata
# Check if trait data is available (i.e., if trait_row is not None)
is_trait_available = trait_row is not None

# Save the initial filtering results
validate_and_save_cohort_info(
    is_final=False,
    cohort=cohort,
    info_path=json_path,
    is_gene_available=is_gene_available,
    is_trait_available=is_trait_available
)

# 4. Clinical Feature Extraction
if trait_row is not None:
    # Extract clinical features
    clinical_df = geo_select_clinical_features(
        clinical_data,  # This should be available from a previous step
        trait=trait,
        trait_row=trait_row,
        convert_trait=convert_trait,
        age_row=age_row,
        convert_age=convert_age,
        gender_row=gender_row,
        convert_gender=convert_gender
    )
    
    # Preview the clinical data
    print("Preview of clinical data:")
    print(preview_df(clinical_df))
    
    # Save clinical data to CSV
    clinical_df.to_csv(out_clinical_data_file)
    print(f"Clinical data saved to {out_clinical_data_file}")


Preview of clinical data:
{'GSM1899888': [0.0], 'GSM1899889': [0.0], 'GSM1899890': [0.0], 'GSM1899891': [1.0], 'GSM1899892': [1.0], 'GSM1899893': [1.0], 'GSM1899894': [0.0], 'GSM1899895': [0.0], 'GSM1899896': [0.0], 'GSM1899897': [0.0], 'GSM1899898': [0.0], 'GSM1899899': [0.0], 'GSM1899900': [0.0], 'GSM1899901': [0.0], 'GSM1899902': [0.0], 'GSM1899903': [0.0], 'GSM1899904': [0.0], 'GSM1899905': [0.0], 'GSM1899906': [0.0], 'GSM1899907': [0.0], 'GSM1899908': [0.0], 'GSM1899909': [0.0], 'GSM1899910': [0.0], 'GSM1899911': [0.0], 'GSM1899912': [0.0], 'GSM1899913': [0.0], 'GSM1899914': [0.0], 'GSM1899915': [0.0], 'GSM1899916': [1.0], 'GSM1899917': [1.0], 'GSM1899918': [1.0], 'GSM1899919': [0.0], 'GSM1899920': [0.0], 'GSM1899921': [0.0], 'GSM1899922': [0.0], 'GSM1899923': [0.0], 'GSM1899924': [0.0], 'GSM1899925': [0.0], 'GSM1899926': [0.0], 'GSM1899927': [0.0], 'GSM1899928': [0.0], 'GSM1899929': [0.0], 'GSM1899930': [0.0], 'GSM1899931': [1.0], 'GSM1899932': [1.0], 'GSM1899933': [1.0], 'GSM189

### Step 3: Gene Data Extraction

In [4]:
# 1. Get the file paths for the SOFT file and matrix file
soft_file, matrix_file = geo_get_relevant_filepaths(in_cohort_dir)

# 2. First, let's examine the structure of the matrix file to understand its format
import gzip

# Peek at the first few lines of the file to understand its structure
with gzip.open(matrix_file, 'rt') as file:
    # Read first 100 lines to find the header structure
    for i, line in enumerate(file):
        if '!series_matrix_table_begin' in line:
            print(f"Found data marker at line {i}")
            # Read the next line which should be the header
            header_line = next(file)
            print(f"Header line: {header_line.strip()}")
            # And the first data line
            first_data_line = next(file)
            print(f"First data line: {first_data_line.strip()}")
            break
        if i > 100:  # Limit search to first 100 lines
            print("Matrix table marker not found in first 100 lines")
            break

# 3. Now try to get the genetic data with better error handling
try:
    gene_data = get_genetic_data(matrix_file)
    print(gene_data.index[:20])
except KeyError as e:
    print(f"KeyError: {e}")
    
    # Alternative approach: manually extract the data
    print("\nTrying alternative approach to read the gene data:")
    with gzip.open(matrix_file, 'rt') as file:
        # Find the start of the data
        for line in file:
            if '!series_matrix_table_begin' in line:
                break
                
        # Read the headers and data
        import pandas as pd
        df = pd.read_csv(file, sep='\t', index_col=0)
        print(f"Column names: {df.columns[:5]}")
        print(f"First 20 row IDs: {df.index[:20]}")
        gene_data = df


Found data marker at line 61
Header line: "ID_REF"	"GSM1899888"	"GSM1899889"	"GSM1899890"	"GSM1899891"	"GSM1899892"	"GSM1899893"	"GSM1899894"	"GSM1899895"	"GSM1899896"	"GSM1899897"	"GSM1899898"	"GSM1899899"	"GSM1899900"	"GSM1899901"	"GSM1899902"	"GSM1899903"	"GSM1899904"	"GSM1899905"	"GSM1899906"	"GSM1899907"	"GSM1899908"	"GSM1899909"	"GSM1899910"	"GSM1899911"	"GSM1899912"	"GSM1899913"	"GSM1899914"	"GSM1899915"	"GSM1899916"	"GSM1899917"	"GSM1899918"	"GSM1899919"	"GSM1899920"	"GSM1899921"	"GSM1899922"	"GSM1899923"	"GSM1899924"	"GSM1899925"	"GSM1899926"	"GSM1899927"	"GSM1899928"	"GSM1899929"	"GSM1899930"	"GSM1899931"	"GSM1899932"	"GSM1899933"	"GSM1899934"	"GSM1899935"	"GSM1899936"	"GSM1899937"	"GSM1899938"	"GSM1899939"
First data line: 1	9.113926239	9.353160006	9.340419788	9.068727157	8.993265236	9.006226326	10.25871409	10.13329143	10.13550261	9.671620081	9.57993636	9.361243542	9.883231123	8.850008948	8.631560726	8.812670809	10.30292328	10.2893897	10.34595556	9.009325384	9.172475557	9.24

Index(['1', '2', '3', '4', '5', '6', '7', '8', '9', '10', '11', '12', '13',
       '14', '15', '16', '17', '18', '19', '20'],
      dtype='object', name='ID')


### Step 4: Gene Identifier Review

In [5]:
# Looking at the identifiers in the gene expression data, I can see they are just numeric indices (1, 2, 3, etc.)
# These are not human gene symbols but rather numeric probe IDs that need to be mapped to gene symbols
# This is typical for microarray data where probes need to be mapped to their corresponding genes

requires_gene_mapping = True


### Step 5: Gene Annotation

In [6]:
# 1. Let's first examine the structure of the SOFT file before trying to parse it
import gzip

# Look at the first few lines of the SOFT file to understand its structure
print("Examining SOFT file structure:")
try:
    with gzip.open(soft_file, 'rt') as file:
        # Read first 20 lines to understand the file structure
        for i, line in enumerate(file):
            if i < 20:
                print(f"Line {i}: {line.strip()}")
            else:
                break
except Exception as e:
    print(f"Error reading SOFT file: {e}")

# 2. Now let's try a more robust approach to extract the gene annotation
# Instead of using the library function which failed, we'll implement a custom approach
try:
    # First, look for the platform section which contains gene annotation
    platform_data = []
    with gzip.open(soft_file, 'rt') as file:
        in_platform_section = False
        for line in file:
            if line.startswith('^PLATFORM'):
                in_platform_section = True
                continue
            if in_platform_section and line.startswith('!platform_table_begin'):
                # Next line should be the header
                header = next(file).strip()
                platform_data.append(header)
                # Read until the end of the platform table
                for table_line in file:
                    if table_line.startswith('!platform_table_end'):
                        break
                    platform_data.append(table_line.strip())
                break
    
    # If we found platform data, convert it to a DataFrame
    if platform_data:
        import pandas as pd
        import io
        platform_text = '\n'.join(platform_data)
        gene_annotation = pd.read_csv(io.StringIO(platform_text), delimiter='\t', 
                                      low_memory=False, on_bad_lines='skip')
        print("\nGene annotation preview:")
        print(preview_df(gene_annotation))
    else:
        print("Could not find platform table in SOFT file")
        
        # Try an alternative approach - extract mapping from other sections
        with gzip.open(soft_file, 'rt') as file:
            for line in file:
                if 'ANNOTATION information' in line or 'annotation information' in line:
                    print(f"Found annotation information: {line.strip()}")
                if line.startswith('!Platform_title') or line.startswith('!platform_title'):
                    print(f"Platform title: {line.strip()}")
            
except Exception as e:
    print(f"Error processing gene annotation: {e}")


Examining SOFT file structure:
Line 0: ^DATABASE = GeoMiame
Line 1: !Database_name = Gene Expression Omnibus (GEO)
Line 2: !Database_institute = NCBI NLM NIH
Line 3: !Database_web_link = http://www.ncbi.nlm.nih.gov/geo
Line 4: !Database_email = geo@ncbi.nlm.nih.gov
Line 5: ^SERIES = GSE73637
Line 6: !Series_title = Prior Knowledge Transfer Across Transcriptional Datasets Using Compositional Statistics [Cell lines]
Line 7: !Series_geo_accession = GSE73637
Line 8: !Series_status = Public on Nov 08 2016
Line 9: !Series_submission_date = Sep 30 2015
Line 10: !Series_last_update_date = Nov 10 2016
Line 11: !Series_pubmed_id = 27353327
Line 12: !Series_summary = Compositional statistics and random gene-sets were used to assign the tumor site of origin and histopathology of 18 epithelial ovarian cancer cell lines
Line 13: !Series_overall_design = In the study presented here, we applied Gene Expression Compositional Assignment (GECA) to epithelial ovarian cell lines, using first a reference li


Gene annotation preview:
{'ID': [1, 2, 3, 4, 5], 'ProbeSetID': ['200000_s_at', '200001_at', '200002_at', '200003_s_at', '200004_at'], 'GeneSymbol': ['PRPF8', 'CAPNS1', 'RPL35', 'RPL28', 'EIF4G2'], 'Array': ['Ovarian Cancer DSA', 'Ovarian Cancer DSA', 'Ovarian Cancer DSA', 'Ovarian Cancer DSA', 'Ovarian Cancer DSA'], 'Annotation Date': ['10-Jan-11', '10-Jan-11', '10-Jan-11', '10-Jan-11', '10-Jan-11'], 'Sequence Type': ['Affymetrix human normalisation control', 'Affymetrix human normalisation control', 'Affymetrix human normalisation control', 'Affymetrix human normalisation control', 'Affymetrix human normalisation control'], 'Ensembl Version': ['release 60', 'release 60', 'release 60', 'release 60', 'release 60'], 'Ensembl Genome Version': ['GRCh37', 'GRCh37', 'GRCh37', 'GRCh37', 'GRCh37'], 'Orientation / Description': ['Sense (Fully Exonic)', 'Sense (Fully Exonic)', 'Sense (Fully Exonic)', 'Sense (Fully Exonic)', 'Sense (Fully Exonic)'], 'No. probes aligned': ['9', '8', '8', '9', '11

### Step 6: Gene Identifier Mapping

In [7]:
# 1. Identify which columns in the gene annotation dataframe correspond to identifiers and symbols
# From the gene annotation preview, we can see:
# - 'ID' column contains numeric identifiers matching gene_data.index
# - 'GeneSymbol' column contains the gene symbols we need

# 2. Create a mapping dataframe from the gene annotation data
mapping_df = gene_annotation[['ID', 'GeneSymbol']]
mapping_df = mapping_df.rename(columns={'GeneSymbol': 'Gene'})
mapping_df = mapping_df.dropna()
mapping_df = mapping_df.astype({'ID': 'str'})  # Convert ID to string to match gene_data index

print("Gene mapping preview (first 5 rows):")
print(mapping_df.head())
print(f"Total mapping entries: {len(mapping_df)}")

# 3. Apply the mapping to convert probe-level measurements to gene expression data
gene_data = apply_gene_mapping(gene_data, mapping_df)

print("\nGene expression data after mapping:")
print(f"Shape: {gene_data.shape}")
print("Sample of genes (first 5):")
print(gene_data.index[:5])

# Save the gene expression data to CSV
gene_data.to_csv(out_gene_data_file)
print(f"Gene expression data saved to {out_gene_data_file}")


Gene mapping preview (first 5 rows):
  ID    Gene
0  1   PRPF8
1  2  CAPNS1
2  3   RPL35
3  4   RPL28
4  5  EIF4G2
Total mapping entries: 120373



Gene expression data after mapping:
Shape: (20172, 52)
Sample of genes (first 5):
Index(['A1BG', 'A1CF', 'A2LD1', 'A2M', 'A2ML1'], dtype='object', name='Gene')


Gene expression data saved to ../../output/preprocess/Endometrioid_Cancer/gene_data/GSE73637.csv


### Step 7: Data Normalization and Linking

In [8]:
# 1. Normalize gene symbols in the gene expression data
normalized_gene_data = normalize_gene_symbols_in_index(gene_data)
print(f"Normalized gene data shape: {normalized_gene_data.shape}")
print("First few genes with their expression values after normalization:")
print(normalized_gene_data.head())

# Save the normalized gene data
os.makedirs(os.path.dirname(out_gene_data_file), exist_ok=True)
normalized_gene_data.to_csv(out_gene_data_file)
print(f"Normalized gene data saved to {out_gene_data_file}")

# 2. Check if trait data is available before proceeding with clinical data extraction
if trait_row is None:
    print("Trait row is None. Cannot extract trait information from clinical data.")
    # Create an empty dataframe for clinical features
    clinical_features = pd.DataFrame()
    
    # Create an empty dataframe for linked data
    linked_data = pd.DataFrame()
    
    # Validate and save cohort info
    validate_and_save_cohort_info(
        is_final=True, 
        cohort=cohort, 
        info_path=json_path, 
        is_gene_available=True, 
        is_trait_available=False,  # Trait data is not available
        is_biased=True,  # Not applicable but required
        df=pd.DataFrame(),  # Empty dataframe
        note="Dataset contains gene expression data but lacks clear trait indicators for Duchenne Muscular Dystrophy status."
    )
    print("Data was determined to be unusable due to missing trait indicators and was not saved")
else:
    try:
        # Get the file paths for the matrix file to extract clinical data
        _, matrix_file = geo_get_relevant_filepaths(in_cohort_dir)
        
        # Get raw clinical data from the matrix file
        _, clinical_raw = get_background_and_clinical_data(matrix_file)
        
        # Verify clinical data structure
        print("Raw clinical data shape:", clinical_raw.shape)
        
        # Extract clinical features using the defined conversion functions
        clinical_features = geo_select_clinical_features(
            clinical_df=clinical_raw,
            trait=trait,
            trait_row=trait_row,
            convert_trait=convert_trait,
            age_row=age_row,
            convert_age=convert_age,
            gender_row=gender_row,
            convert_gender=convert_gender
        )
        
        print("Clinical features:")
        print(clinical_features)
        
        # Save clinical features to file
        os.makedirs(os.path.dirname(out_clinical_data_file), exist_ok=True)
        clinical_features.to_csv(out_clinical_data_file)
        print(f"Clinical features saved to {out_clinical_data_file}")
        
        # 3. Link clinical and genetic data
        linked_data = geo_link_clinical_genetic_data(clinical_features, normalized_gene_data)
        print(f"Linked data shape: {linked_data.shape}")
        print("Linked data preview (first 5 rows, first 5 columns):")
        print(linked_data.iloc[:5, :5])
        
        # 4. Handle missing values
        print("Missing values before handling:")
        print(f"  Trait ({trait}) missing: {linked_data[trait].isna().sum()} out of {len(linked_data)}")
        if 'Age' in linked_data.columns:
            print(f"  Age missing: {linked_data['Age'].isna().sum()} out of {len(linked_data)}")
        if 'Gender' in linked_data.columns:
            print(f"  Gender missing: {linked_data['Gender'].isna().sum()} out of {len(linked_data)}")
        
        gene_cols = [col for col in linked_data.columns if col not in [trait, 'Age', 'Gender']]
        print(f"  Genes with >20% missing: {sum(linked_data[gene_cols].isna().mean() > 0.2)}")
        print(f"  Samples with >5% missing genes: {sum(linked_data[gene_cols].isna().mean(axis=1) > 0.05)}")
        
        cleaned_data = handle_missing_values(linked_data, trait)
        print(f"Data shape after handling missing values: {cleaned_data.shape}")
        
        # 5. Evaluate bias in trait and demographic features
        is_trait_biased = False
        if len(cleaned_data) > 0:
            trait_biased, cleaned_data = judge_and_remove_biased_features(cleaned_data, trait)
            is_trait_biased = trait_biased
        else:
            print("No data remains after handling missing values.")
            is_trait_biased = True
        
        # 6. Final validation and save
        is_usable = validate_and_save_cohort_info(
            is_final=True, 
            cohort=cohort, 
            info_path=json_path, 
            is_gene_available=True, 
            is_trait_available=True, 
            is_biased=is_trait_biased, 
            df=cleaned_data,
            note="Dataset contains gene expression data comparing Duchenne muscular dystrophy vs healthy samples."
        )
        
        # 7. Save if usable
        if is_usable and len(cleaned_data) > 0:
            os.makedirs(os.path.dirname(out_data_file), exist_ok=True)
            cleaned_data.to_csv(out_data_file)
            print(f"Linked data saved to {out_data_file}")
        else:
            print("Data was determined to be unusable or empty and was not saved")
            
    except Exception as e:
        print(f"Error processing data: {e}")
        # Handle the error case by still recording cohort info
        validate_and_save_cohort_info(
            is_final=True, 
            cohort=cohort, 
            info_path=json_path, 
            is_gene_available=True, 
            is_trait_available=False,  # Mark as not available due to processing issues
            is_biased=True, 
            df=pd.DataFrame(),  # Empty dataframe
            note=f"Error processing data: {str(e)}"
        )
        print("Data was determined to be unusable and was not saved")

Normalized gene data shape: (20036, 52)
First few genes with their expression values after normalization:
         GSM1899888  GSM1899889  GSM1899890  GSM1899891  GSM1899892  \
Gene                                                                  
A1BG       5.263569    5.598553    5.263475    6.084976    5.997425   
A1CF       2.647713    2.383580    2.605635    2.587824    2.507772   
A2M        6.292947    6.771711    5.949791    6.474002    6.744352   
A2ML1      3.756357    2.472887    3.527244    2.165761    2.767217   
A3GALT2    6.355374    6.217401    6.484687    6.666866    6.405594   

         GSM1899893  GSM1899894  GSM1899895  GSM1899896  GSM1899897  ...  \
Gene                                                                 ...   
A1BG       6.125724    5.257894    5.212058    5.444705    6.021699  ...   
A1CF       3.821882    3.124388    2.756973    3.170520    2.728037  ...   
A2M        6.833200    7.674972    7.644879    7.283829    6.150173  ...   
A2ML1      2.353

Normalized gene data saved to ../../output/preprocess/Endometrioid_Cancer/gene_data/GSE73637.csv


Raw clinical data shape: (4, 53)
Clinical features:
                     GSM1899888  GSM1899889  GSM1899890  GSM1899891  \
Endometrioid_Cancer         0.0         0.0         0.0         1.0   

                     GSM1899892  GSM1899893  GSM1899894  GSM1899895  \
Endometrioid_Cancer         1.0         1.0         0.0         0.0   

                     GSM1899896  GSM1899897  ...  GSM1899930  GSM1899931  \
Endometrioid_Cancer         0.0         0.0  ...         0.0         1.0   

                     GSM1899932  GSM1899933  GSM1899934  GSM1899935  \
Endometrioid_Cancer         1.0         1.0         0.0         0.0   

                     GSM1899936  GSM1899937  GSM1899938  GSM1899939  
Endometrioid_Cancer         0.0         0.0         0.0         0.0  

[1 rows x 52 columns]
Clinical features saved to ../../output/preprocess/Endometrioid_Cancer/clinical_data/GSE73637.csv
Linked data shape: (52, 20037)
Linked data preview (first 5 rows, first 5 columns):
            Endometri

Data shape after handling missing values: (52, 20037)
For the feature 'Endometrioid_Cancer', the least common label is '1.0' with 9 occurrences. This represents 17.31% of the dataset.
The distribution of the feature 'Endometrioid_Cancer' in this dataset is fine.



Linked data saved to ../../output/preprocess/Endometrioid_Cancer/GSE73637.csv
